In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import pandas as pd
import chainer

### pytorch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import time

In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### PTB Data
#### Question 1: Should the text be broken down to samples split on "eos" token?
YES
#### Question 2: Should the text be prepended with a sos token?
YES

In [5]:
# unpacking the data from chainer
train, val, test = chainer.datasets.get_ptb_words()

In [6]:
# the data is already separated into a numpy array
print(f"train data: {type(train)} {train.shape} {train}")
print(f"val data: {type(val)} {val.shape} {val}")
print(f"test data: {type(test)} {test.shape} {test}")

train data: <class 'numpy.ndarray'> (929589,) [ 0  1  2 ... 39 26 24]
val data: <class 'numpy.ndarray'> (73760,) [2211  396 1129 ...  108   27   24]
test data: <class 'numpy.ndarray'> (82430,) [142  78  54 ...  87 214  24]


In [7]:
# ptb_dict is a dictionary containing words (key) to idx (value)
vocab2idx = chainer.datasets.get_ptb_words_vocabulary()
vocab2idx = {k:v for k, v in vocab2idx.items()}

#NOTE: PAD = 10000, <sos> = 10001, <m> = 10002
vocab2idx['PAD'] = len(vocab2idx)
vocab2idx['<sos>'] = 10001
vocab2idx['<m>'] = 10002

#creating a reverse dict to turn an index back into word for sanity check
idx2vocab = {v:k for k, v in vocab2idx.items()}
print(f"Number of vocabulary: {len(vocab2idx)}")

Number of vocabulary: 10003


In [8]:
def split_sentence(data):
    """
    This function splits the text data into individual sentences split on the <eos> token
    and prepends the <sos> token in the front.
    """
    samples, sentence, eos_idx = [], [vocab2idx['<sos>']], vocab2idx['<eos>']
    for idx in data:
        if idx != eos_idx:  #25 is the idx for the <eos> token
            sentence.append(idx)
        else:
            sentence.append(idx)
            samples.append(sentence)
            sentence = [vocab2idx['<sos>']]
    return samples

In [9]:
#splitting each sequence as an individual sample
train_samples = split_sentence(train)
val_samples = split_sentence(val)
test_samples = split_sentence(test)

In [10]:
sentence = [idx2vocab[idx] for idx in train_samples[5]]
print(' '.join(sentence))

<sos> the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said <eos>


In [11]:
#val_samples sequence
sentence = [idx2vocab[idx] for idx in val_samples[5]]
print(' '.join(sentence))

<sos> eventually viewers may grow <unk> with the technology and <unk> the cost <eos>


In [12]:
#test_samples sequence
sentence = [idx2vocab[idx] for idx in test_samples[5]]
print(' '.join(sentence))

<sos> heavy selling of standard & poor 's 500-stock index futures in chicago <unk> beat stocks downward <eos>


In [13]:
def mask_tokens(sentence, mask_prob, sub_prob):
    mask_sent, count = [], int(len(sentence)*sub_prob)
    for idx, token in enumerate(sentence):
        if np.random.uniform() < mask_prob and token not in [vocab2idx['<sos>'], vocab2idx['<eos>']]:
            mask_sent.append(vocab2idx['<m>'])
            count -= 1
        else:
            mask_sent.append(token)
        if count == 0:
            return mask_sent + sentence[idx+1:]
    return mask_sent

In [14]:
# sanity check that the masking worked
len(mask_tokens(train_samples[0], .2, .2)), len(train_samples[0])

(26, 26)

In [15]:
class PTBDataset(Dataset):
    """
    Setting up the Penn Tree Bank Dataset.
    """
    def __init__(self, X, mask_prob, sub_prob):
        self.X = X
        self.masked_X = [mask_tokens(x, mask_prob, sub_prob) for x in X]
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.LongTensor(self.X[idx]), torch.LongTensor(self.masked_X[idx])

In [16]:
# calling on the dataset
train_ds = PTBDataset(train_samples, .2, .2)
val_ds = PTBDataset(val_samples, .2, .2)
test_ds = PTBDataset(test_samples, .2, .2)

In [17]:
def collate(batch):
    """
    NOTE: batch without any labels is just a list of tensors coming from Dataset.
    padding each sequence.
    """
    (X, mask_X) = zip(*batch)
    x_len = [len(x) for x in mask_X]
    mask_x_pad = pad_sequence(mask_X, batch_first=True, padding_value=10000)
    x_pad = pad_sequence(X, batch_first=True, padding_value=10000)
    return mask_x_pad, x_pad, x_len

In [18]:
batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate)
valid_dl = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate)
test_dl = DataLoader(test_ds, batch_size=batch_size, collate_fn=collate)

### Generator
#### Figure out how does bidirectional go with all of this? #### How to pass in bidirectional encoder vectors into a unidirectional decoder?
* The encoder will generate hidden states that are of size 2*hidden_dimension (Encoder) size. I have to concatenate the two vectors of the hidden state. The cell state isnt really needed per Yannet but could still be used. The decoder hidden dimension size needs to be the same as the encoder. The concatenation should be done in the Seq2Seq.
   
#### Figure out how to do the masking for each sentence. How many tokens do we mask? Do they have to be in sequential order?
* This was done setting a prob p of masking each token. The max number of tokens to mask is also set of a proportion length of the text. As a result, they were not done in sequential order.
#### How to setup the Seq2Seq to generate text?
* Perhaps it may be easier to go with Yannet's setup than benvrett? Need to discuss with Shirkar.

#### Getting this error
* AttributeError: 'int' object has no attribute 'backward'. The weird thing is that it trains for a few epochs.

In [19]:
class GenEncoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(.5)
        
    def forward(self, x, lengths):
        x = self.dropout(self.emb(x))
        x_pack = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out, (hidden, cell) = self.lstm(x_pack)  #NOTE: If (h_0, c_0) is not provided, both h_0 and c_0 default to zero.
        return hidden, cell

In [20]:
class GenDecoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, h0, c0):
        x = self.dropout(self.emb(x))
        out, (hidden, cell) = self.lstm(x, (h0, c0))  #passing in the initial hidden state and cell state
        return self.linear(hidden[-1]), hidden, cell

In [21]:
mask_x, x, lengths = next(iter(train_dl))
# mask_x, x, lengths

In [22]:
encoder = GenEncoder(5, 10, len(vocab2idx))
contexts, cells = encoder(mask_x, lengths)
contexts.size(), cells.size()  # the context and cell tensors are shape: (n_layers*n_dir, batch_size, hidden_dim)

(torch.Size([2, 64, 10]), torch.Size([2, 64, 10]))

In [23]:
flatten_contexts, flatten_cells = torch.flatten(contexts.transpose(1,0), 1), torch.flatten(cells.transpose(1,0), 1)
flatten_contexts.size(), flatten_cells.size()  # after concatenating

(torch.Size([64, 20]), torch.Size([64, 20]))

In [24]:
flatten_contexts.unsqueeze(0).size()  #unsqueeze(0) to make shape: (n_layers*n_dir, batch_size, hidden_dim)

torch.Size([1, 64, 20])

In [25]:
mask_x[:, 2]

tensor([   32, 10002,   119,  5728,    42,  6351,   811,  2059,  1972,  1401,
           32,  7360,  1107,    98,  8283,   383,    26,    27,  2968, 10002,
         1457,  8854,  2281,  1094,  4417,    32,   169,    98,  1042, 10002,
           26,    54,   131,    32,  2312,    35,  7965,    40,  8815,  1802,
           42,    26,  2264,   159,   744,    40,  1333,   247,  1420,  9765,
          296,    93,  3699,   687, 10002,   718,  1321,    27, 10002,   144,
          125,  2011,   124,  7163])

In [26]:
decoder = GenDecoder(20, 20, len(vocab2idx))
out, hidden, cell = decoder(x, flatten_contexts.unsqueeze(0), flatten_cells.unsqueeze(0))

In [27]:
x.size()

torch.Size([64, 77])

In [28]:
hidden.size(), cell.size(), out.size()

(torch.Size([1, 64, 20]), torch.Size([1, 64, 20]), torch.Size([64, 10003]))

In [29]:
out.size()

torch.Size([64, 10003])

In [30]:
out.argmax(dim=1)

tensor([ 494, 7085, 2859, 4812, 9081, 2953, 4312, 9570, 4215, 1740, 8942, 2859,
        4215, 2859, 4812, 2859, 2859, 8260, 5443, 5443, 5443, 4812, 5443, 4802,
        3427, 6872,  494, 6872, 6872, 9081, 5443, 5443, 2859, 9081, 5005, 2611,
        5702, 8920, 4215, 1740, 4812, 7085, 7733, 3674, 4659, 4659, 5443, 2859,
        7699, 3427, 2859,  494, 6248,  494, 2859, 3427, 5702,  494, 1740, 6872,
        9081, 3228, 7085, 2859], grad_fn=<NotImplemented>)

In [31]:
mask_x[:, 6].unsqueeze(1).size()

torch.Size([64, 1])

In [32]:
x[:,0].unsqueeze(1).size()

torch.Size([64, 1])

In [33]:
torch.LongTensor([1,2,3])

tensor([1, 2, 3])

In [34]:
def batch(encoder, decoder, enc_optimizer, dec_optimizer, mask_x, x, lengths, train=True):
    if train:
        encoder.train()
        decoder.train()
    else:
        encoder.eval()
        decoder.eval()
    # zero grad for both optimizers
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    loss = 0
    
    # **ENCODER**
    #passing the masked tokens into the encoder to retrieve context vectors and final cells
    contexts, cells = encoder(mask_x, lengths)  # context, cell shape: (n_layers*n_dir, batch_size, hidden_dim)
    
    #first concatenate the bidirectional hidden & cell states into one context & cell tensors
    #unsqueeze(0) to shape: (n_layers*n_dir, batch_size, hidden_dim)
    hidden = torch.flatten(contexts.transpose(1,0), 1).unsqueeze(0)
    cell = torch.flatten(cells.transpose(1,0), 1).unsqueeze(0) 
    
    # **DECODER**
    batch_size = mask_x.size(0)  #batch_size
    batch_target_length = mask_x.size(1)  # this target length is the max seq length of batch_size
    decoder_input  = mask_x[:, 0].unsqueeze(1)  #unsqueeze to make sure its still batch_size, idx_dim
    
    for idx in range(1, batch_target_length):
        output, hidden, cell = decoder(decoder_input, hidden, cell)
        x_idx = x[:,idx]   # dont think .unsqueeze(1) is necessary
        
        #if (x_idx.eq(10000)).sum() > 0:  <- discuss with shrikar whether this is necessary
            # ignore the padding index so it doesnt count towards the loss
        loss += F.cross_entropy(output, x_idx, ignore_index = 10000)
        
        # setting up for the next input USE torch.where!!!!
        decoder_input = torch.where(mask_x[:,idx].eq(10002), output.argmax(dim=1), x[:,idx]).unsqueeze(1)
        
    # updating the gradient
    if train:
        loss.backward()  #one loss for both optimizers?
        enc_optimizer.step()
        dec_optimizer.step()
    return loss.item()

In [35]:
def train_model(encoder, decoder, enc_optimizer, dec_optimizer, train_dl, val_dl, epochs=10):
    for epoch in range(epochs):
        start = time.time()
        total, total_loss = 0, 0
        total_v = 0
        encoder.train()
        decoder.train()
        val_loss = 0
        for mask_x, x, lengths in train_dl:
            loss = batch(encoder, decoder, enc_optimizer, dec_optimizer, mask_x.to(device), x.to(device), lengths)
            total_loss += loss*mask_x.size(0)
            total += mask_x.size(0)
        for mask_x, x, lengths in val_dl:
            v_loss = batch(encoder, decoder, enc_optimizer, dec_optimizer, mask_x.to(device), x.to(device), lengths, False)
            val_loss += v_loss*mask_x.size(0)
            total_v += mask_x.size(0)
        print(f"Epoch {epoch+1}  Training Loss: {total_loss/total:.3f} Val Loss: {val_loss/total_v:.3f} Time: {time.time()-start}")

### Model Training

In [36]:
encoder = GenEncoder(5, 10, len(vocab2idx)).to(device)
decoder = GenDecoder(20, 20, len(vocab2idx)).to(device)
enc_optimizer = optim.Adam(encoder.parameters(), lr=.001)
dec_optimizer = optim.Adam(decoder.parameters(), lr=.001)

In [37]:
batch_size = 256
train_ds = PTBDataset(train_samples, .2, .2)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate)
val_ds = PTBDataset(val_samples, .2, .2)
val_dl = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate)

In [38]:
train_model(encoder, decoder, enc_optimizer, dec_optimizer, train_dl, val_dl, 10)

Epoch 1  Training Loss: 476.187 Val Loss: 376.954 Time: 23.552446365356445
Epoch 2  Training Loss: 377.950 Val Loss: 368.877 Time: 22.762675523757935
Epoch 3  Training Loss: 374.850 Val Loss: 368.036 Time: 21.881603956222534
Epoch 4  Training Loss: 371.566 Val Loss: 366.539 Time: 24.28552222251892
Epoch 5  Training Loss: 374.652 Val Loss: 363.456 Time: 24.688134908676147
Epoch 6  Training Loss: 369.885 Val Loss: 360.547 Time: 24.511171579360962
Epoch 7  Training Loss: 364.771 Val Loss: 357.915 Time: 24.20044207572937
Epoch 8  Training Loss: 365.724 Val Loss: 355.696 Time: 24.626968145370483
Epoch 9  Training Loss: 362.720 Val Loss: 353.842 Time: 24.9388325214386
Epoch 10  Training Loss: 359.964 Val Loss: 351.956 Time: 24.538982391357422


<br>